In [2]:
import sympy as sym
import numpy as np
np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})
import matplotlib.pyplot as plt

In [3]:
def neglect_terms(expr, N):
    # Function to neglect terms of order N and higher in a polynomial
    # Split the expression into numerator and denominator
    num, denom = expr.as_numer_denom()
    # Process numerator and denominator to remove terms of order N and higher
    num_neglected = sum(coeff * term for term, coeff in num.as_coefficients_dict().items() if all(pow < N for var, pow in term.as_powers_dict().items()))
    denom_neglected = sum(coeff * term for term, coeff in denom.as_coefficients_dict().items() if all(pow < N for var, pow in term.as_powers_dict().items()))
    # Reconstruct the fraction
    return num_neglected / denom_neglected

def neglect_terms_in_matrix(matrix, N):
    # Process each element in the matrix
    processed_matrix = matrix.applyfunc(lambda x: neglect_terms(x, N))

    return processed_matrix

In [4]:
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

In [5]:
C_mat = sym.Matrix([[C_R+C_C/2 , 0         , -C_C/2    , +C_C/2     ],
                    [0         , C_F+C_C/2 , -C_C/2    , +C_C/2     ],
                    [-C_C/2    , +C_C/2    , C_R+C_C/2 , 0          ],
                    [-C_C/2    , +C_C/2    , 0         , C_F+C_C/2  ] ])

In [14]:
C_mat = sym.Matrix([[C_R+C_C/2 , 0         , -C_C/2    , 0         ],
                     [0        , C_F+C_C/2 , 0         , +C_C/2    ],
                     [-C_C/2   , 0         , C_R+C_C/2 , 0         ],
                     [0        , +C_C/2    , 0         , C_F+C_C/2 ] ])

In [38]:
C_inv = C_mat**-1

In [39]:
sym.simplify(C_inv)

Matrix([
[2*(C_C**3 + 6*C_C**2*C_F + 4*C_C*C_F**2 + 8*C_C*C_F*C_R + 8*C_F**2*C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                                                   4*C_C**2*(-C_F + C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                                   2*C_C*(C_C**2 + 4*C_C*C_F + 4*C_F**2)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2),                     2*C_C*(-C_C**2 - 2*C_C*C_F - 2*C_C*C_R - 4*C_F*C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2)],
[                                                   4*C_C**2*(C_F - C_R)/(C_C**4 + 4*C_C**3*C_F + 4*C_C**3*C_R + 24*C_C**2*C_F*C_R + 16*C_C*C_F**2*C_R + 16*C_C*C_F*C_R**2 + 16*C_F**2*C_R**2), 2*(C_C**3 + 6*C_C**2*C_R + 8*C_C*

In [9]:
Csh = 15
C   = 15
CJ  = 3
Cc  = 0.1

In [10]:
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)
C_inv_sym_to_num = np.array( C_inv.subs({C_C:Cc, C_R:C/2, C_F:C/2 + Csh + CJ}).evalf() )

NameError: name 'C_inv' is not defined

In [16]:
C_R = C/2
C_C = Cc
C_F = C/2 + Csh + CJ

𐌲_R = C / (C_R+C_C/2)
𐌲_F = C / (C_F+C_C/2)
ε = C_C/2 / C

C_inv_approx = 1/C * np.array([[ 𐌲_R    ,   0       ,  ε*𐌲_R**2,  -ε*𐌲_R**2],
                               [0       ,   𐌲_F     ,  ε*𐌲_F**2,  -ε*𐌲_F**2],
                               [ε*𐌲_R**2,  -ε*𐌲_R**2,   𐌲_R    ,   0       ],
                               [ε*𐌲_F**2,  -ε*𐌲_F**2,   0      ,   𐌲_F     ]])

# C_inv_approx = 1/C * np.array([[ 𐌲_R    ,   0       ,  ε*𐌲_R**2,   0         ],
#                                [0       ,   𐌲_F     ,      0  ,  -ε*𐌲_F**2  ],
#                                [ε*𐌲_R**2,   0       ,   𐌲_R    ,   0         ],
#                                [0       ,  -ε*𐌲_F**2,   0      ,   𐌲_F       ]])

In [17]:
C_F / C_R

3.4

In [18]:
C_num = np.array  ([[C_R+C_C/2, 0      , -C_C/2 , +C_C/2   ],
                    [0      , C_F+C_C/2, -C_C/2 , +C_C/2   ],
                    [-C_C/2 , +C_C/2 , C_R+C_C/2, 0        ],
                    [-C_C/2 , +C_C/2 , 0      , C_F+C_C/2] ])
# C_num = np.array  ([[C_R+C_C/2, 0      , -C_C/2 ,0       ],
#                     [0      , C_F+C_C/2, 0      , +C_C/2 ],
#                     [-C_C/2 , 0      , C_R+C_C/2, 0      ],
#                     [0      , +C_C/2 , 0      , C_F+C_C/2] ])

In [19]:
np.linalg.inv(C_num)

array([[1.325e-01, -1.209e-06, 8.772e-04, -2.592e-04],
       [1.209e-06, 3.914e-02, 2.592e-04, -7.659e-05],
       [8.772e-04, -2.592e-04, 1.325e-01, -1.209e-06],
       [2.592e-04, -7.659e-05, 1.209e-06, 3.914e-02]])

In [20]:
(1 + (1 - C_R/C) + (1 - C_R/C)**2)/C

0.11666666666666667

In [21]:
(1 + (1 - C_F/C) + (1 - C_F/C)**2)/C

0.05266666666666667

In [23]:
-2*ε*(1-C_R/C)**2 /C

-0.00011111111111111112

In [27]:
(ε*(1-C_R/C) + ε*(1-C_F/C) )/C

-4.444444444444445e-05

In [222]:
C_inv_approx

array([[1.325e-01, 0.000e+00, 8.772e-04, -8.772e-04],
       [0.000e+00, 1.325e-01, 8.772e-04, -8.772e-04],
       [8.772e-04, -8.772e-04, 1.325e-01, 0.000e+00],
       [8.772e-04, -8.772e-04, 0.000e+00, 1.325e-01]])

In [214]:
ε**2

1.1111111111111113e-05

In [217]:
0.001**2

1e-06

In [10]:
# Define the symbols
alpha= sym.symbols('\\alpha', real=True, positive = True, nonzero=True)
N=6
C_mat = sym.Matrix(N,N, lambda i, j: -1 if abs(i-j) == 2 else 0)
C_mat**2

Matrix([
[1, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 1],
[0, 0, 2, 0, 0, 0],
[0, 0, 0, 2, 0, 0],
[1, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 1]])

In [149]:
# Define the symbols
alpha= sym.symbols('\\alpha', real=True, positive = True, nonzero=True)
N=6
C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2 else -alpha if ((i==N-1 and j==0) or (i==0 and j==N-1) )  else 0)
# C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2 else -alpha if abs(i-j) == 1 else -alpha if ((i==N-1 and j==0) or (i==0 and j==N-1) )  else 0)
# C_mat = sym.Matrix(N,N, lambda i, j: 1+alpha if i == j else -alpha if abs(i-j) == 2  else 0)

C_mat = sym.Matrix([[1+alpha, 0      , -alpha , +alpha ],
                    [0      , 2+alpha, -alpha , +alpha ],
                    [-alpha , +alpha ,1+alpha, 0      ],
                    [-alpha , +alpha , 0      , 2+alpha] ])

C_mat

Matrix([
[\alpha + 1,          0,    -\alpha,     \alpha],
[         0, \alpha + 2,    -\alpha,     \alpha],
[   -\alpha,     \alpha, \alpha + 1,          0],
[   -\alpha,     \alpha,          0, \alpha + 2]])

In [150]:
C_inv = C_mat **-1
# C_inv = sym.simplify(C_inv)

In [151]:
C_inv

Matrix([
[(\alpha**3 + 6*\alpha**2 + 8*\alpha + 4)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),                               -\alpha**2/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),     (\alpha**3 + 4*\alpha**2 + 4*\alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),    (-\alpha**3 - 3*\alpha**2 - 2*\alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4)],
[                               \alpha**2/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4), (\alpha**3 + 3*\alpha**2 + 5*\alpha + 2)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),     (\alpha**3 + 3*\alpha**2 + 2*\alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),      (-\alpha**3 - 2*\alpha**2 - \alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4)],
[    (\alpha**3 + 4*\alpha**2 + 4*\alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 4),    (-\alpha**3 - 3*\alpha**2 - 2*\alpha)/(\alpha**4 + 6*\alpha**3 + 12*\alpha**2 + 12*\alpha + 

In [152]:
neglect_terms_in_matrix(C_inv,2)

Matrix([
[(8*\alpha + 4)/(12*\alpha + 4),                              0,       4*\alpha/(12*\alpha + 4),      -2*\alpha/(12*\alpha + 4)],
[                             0, (5*\alpha + 2)/(12*\alpha + 4),       2*\alpha/(12*\alpha + 4),        -\alpha/(12*\alpha + 4)],
[      4*\alpha/(12*\alpha + 4),      -2*\alpha/(12*\alpha + 4), (8*\alpha + 4)/(12*\alpha + 4),                              0],
[      2*\alpha/(12*\alpha + 4),        -\alpha/(12*\alpha + 4),                              0, (5*\alpha + 2)/(12*\alpha + 4)]])

In [9]:
U_0   = sym.polys.orthopolys.chebyshevu_poly(0,   x=(alpha+1)/(2*alpha)).simplify()
U_N   = sym.polys.orthopolys.chebyshevu_poly(N,   x=(alpha+1)/(2*alpha)).simplify()
U_Np1 = sym.polys.orthopolys.chebyshevu_poly(N+1, x=(alpha+1)/(2*alpha)).simplify()
U_Np2 = sym.polys.orthopolys.chebyshevu_poly(N+2, x=(alpha+1)/(2*alpha)).simplify()

In [17]:
C_inv[0,0]

(-2*\alpha**4 + \alpha**3 + 7*\alpha**2 + 5*\alpha + 1)/(\alpha**6 - 2*\alpha**5 - 9*\alpha**4 + 10*\alpha**2 + 6*\alpha + 1)

In [10]:
C_inv_diag = U_Np2**-1 * alpha ** -1 * U_0 * U_Np1
C_inv_diag.expand().simplify()

(\alpha**7 + 3*\alpha**6 - 9*\alpha**5 - 15*\alpha**4 + 5*\alpha**3 + 15*\alpha**2 + 7*\alpha + 1)/(6*\alpha**7 + 3*\alpha**6 - 24*\alpha**5 - 20*\alpha**4 + 14*\alpha**3 + 21*\alpha**2 + 8*\alpha + 1)

In [11]:
C_inv_sub_diag = U_Np2**-1 * alpha ** -1 * U_0 * U_N
C_inv_sub_diag.expand().simplify()

\alpha*(\alpha**6 - 2*\alpha**5 - 9*\alpha**4 + 10*\alpha**2 + 6*\alpha + 1)/(6*\alpha**7 + 3*\alpha**6 - 24*\alpha**5 - 20*\alpha**4 + 14*\alpha**3 + 21*\alpha**2 + 8*\alpha + 1)

In [6]:
# Define the symbols
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

# Create a 6x6 matrix based on the given pattern
C_mat = sym.Matrix(4,4, lambda i, j: C_R+C_C/2 if (i == j and np.mod(i,2)==0) else C_F+C_C/2 if (i == j and np.mod(i,2)!=0)  else -C_C/2 if abs(i-j) == 2 else 0)
C_mat[1,-1] *=-1
C_mat[-1,1] *=-1
C_mat

Matrix([
[C_C/2 + C_R,           0,      -C_C/2,           0],
[          0, C_C/2 + C_F,           0,       C_C/2],
[     -C_C/2,           0, C_C/2 + C_R,           0],
[          0,       C_C/2,           0, C_C/2 + C_F]])

In [7]:
C_inv = C_mat **-1

In [8]:
sym.simplify(C_inv)

Matrix([
[(C_C/2 + C_R)/(C_R*(C_C + C_R)),                               0,         C_C/(2*C_R*(C_C + C_R)),                               0],
[                              0, (C_C/2 + C_F)/(C_F*(C_C + C_F)),                               0,        -C_C/(2*C_F*(C_C + C_F))],
[        C_C/(2*C_R*(C_C + C_R)),                               0, (C_C/2 + C_R)/(C_R*(C_C + C_R)),                               0],
[                              0,        -C_C/(2*C_F*(C_C + C_F)),                               0, (C_C/2 + C_F)/(C_F*(C_C + C_F))]])

In [30]:
sym.print_latex(sym.simplify(C_inv))

\left[\begin{matrix}\frac{\frac{C_{C}}{2} + C_{R}}{C_{R} \left(C_{C} + C_{R}\right)} & 0 & \frac{C_{C}}{2 C_{R} \left(C_{C} + C_{R}\right)} & 0\\0 & \frac{\frac{C_{C}}{2} + C_{F}}{C_{F} \left(C_{C} + C_{F}\right)} & 0 & \frac{C_{C}}{2 C_{F} \left(C_{C} + C_{F}\right)}\\\frac{C_{C}}{2 C_{R} \left(C_{C} + C_{R}\right)} & 0 & \frac{\frac{C_{C}}{2} + C_{R}}{C_{R} \left(C_{C} + C_{R}\right)} & 0\\0 & \frac{C_{C}}{2 C_{F} \left(C_{C} + C_{F}\right)} & 0 & \frac{\frac{C_{C}}{2} + C_{F}}{C_{F} \left(C_{C} + C_{F}\right)}\end{matrix}\right]


In [18]:
# Define the symbols
C_R, C_F, C_C = sym.symbols('C_R, C_F, C_C', real=True, positive=True, nonzero=True)

# Create a 6x6 matrix based on the given pattern
C_mat = sym.Matrix(10,10, lambda i, j: C_R+C_C/2 if (i == j and np.mod(i,2)==0) else C_F+C_C/2 if (i == j and np.mod(i,2)!=0)  else -C_C/2 if abs(i-j) == 2 else 0)

C_mat

Matrix([
[C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0,           0,           0,           0,           0],
[          0, C_C/2 + C_F,           0,      -C_C/2,           0,           0,           0,           0,           0,           0],
[     -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0,           0,           0],
[          0,      -C_C/2,           0, C_C/2 + C_F,           0,      -C_C/2,           0,           0,           0,           0],
[          0,           0,      -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0,           0,           0],
[          0,           0,           0,      -C_C/2,           0, C_C/2 + C_F,           0,      -C_C/2,           0,           0],
[          0,           0,           0,           0,      -C_C/2,           0, C_C/2 + C_R,           0,      -C_C/2,           0],
[          0,           0,           0,           0,           0,  

In [19]:
C_inv = C_mat **-1

In [20]:
sym.simplify(C_inv)

Matrix([
[(-C_C**4/4 - C_C**3*C_R + 3*C_C**2*C_R**2 + 8*C_C*C_R**3 + 4*C_R**4)/(C_R*(-C_C**4 - C_C**3*C_R + 6*C_C**2*C_R**2 + 10*C_C*C_R**3 + 4*C_R**4)),                                                                                                                                              0,                C_C*(-C_C**2 + 4*C_C*C_R + 4*C_R**2)/(4*C_R*(-C_C**3 + C_C**2*C_R + 4*C_C*C_R**2 + 2*C_R**3)),                                                                                                            0,            C_C**2/(-C_C**3 + 6*C_C*C_R**2 + 4*C_R**3),                                                     0,                                              C_C**3/(4*C_R*(-C_C**3 + C_C**2*C_R + 4*C_C*C_R**2 + 2*C_R**3)),                                                                                                            0,                                                             C_C**4/(4*C_R*(-C_C**4 - C_C**3*C_R + 6*C_C**2*C_R**2 + 10*C_C*C_R**3 + 4*C_R**4)),         